# Precheck

With this template, you can find errors in your strategy before submitting.

1) Click menu "Kernel" -> "Restart Kernel and Run All Cells…"

2) Compare the strategy statistics with the results in strategy.ipynb

In [2]:
from qnt.precheck import *

data_type = 'stocks_nasdaq100'  # 'futures', 'stocks', 'futures', 'cryptofutures'

run_init()
# Runs your strategy multiple times cutting the data tail.
# It is similar to the step-by-step calculation.
evaluate_passes(passes=3, data_type=data_type)
weights = assemble_output()

Run init.ipynb..
cmd:  jupyter nbconvert --to html --ExecutePreprocessor.timeout=1800 --execute init.ipynb --stdout | html2text -utf8
output:
usage: html2text [-h] [--default-image-alt DEFAULT_IMAGE_ALT] [--pad-tables]
                 [--no-wrap-links] [--wrap-list-items] [--wrap-tables]
                 [--ignore-emphasis] [--reference-links] [--ignore-links]
                 [--ignore-mailto-links] [--protect-links] [--ignore-images]
                 [--images-as-html] [--images-to-alt] [--images-with-size]
                 [-g] [-d] [-e] [-b BODY_WIDTH] [-i LIST_INDENT] [-s]
                 [--escape-all] [--bypass-tables] [--ignore-tables]
                 [--single-line-break] [--unicode-snob] [--no-automatic-links]
                 [--no-skip-internal-links] [--links-after-para] [--mark-code]
                 [--decode-errors DECODE_ERRORS] [--open-quote OPEN_QUOTE]
                 [--close-quote CLOSE_QUOTE] [--version] [--include-sup-sub]
                 [filename] [encodin

/ |#                                                  | 0 Elapsed Time: 0:00:00
| |#                                               | 6229 Elapsed Time: 0:00:00


fetched chunk 1/1 6s
Data loaded 6s
There are no missed dates.
---
Evaluation complete.
Merge outputs...
init output: precheck_results/2006-01-04.fractions.nc.gz 2006-01-04 2006-01-04
add output: precheck_results/2015-07-08.fractions.nc.gz 2015-07-08 2015-07-08
add output: precheck_results/2025-01-08.fractions.nc.gz 2025-01-08 2025-01-08


In [3]:
# run this code for your strategy in strategy.ipnb and compare the results
import qnt.data as qndata
import qnt.stats as qns
import qnt.graph as qngraph

data = qndata.stocks.load_ndx_data(min_date="2006-01-01")
stats = qns.calc_stat(data, weights.sel(time=slice("2006-01-01", None)))
display(stats.to_pandas().tail())

performance = stats.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

fetched chunk 1/7 0s
fetched chunk 2/7 0s
fetched chunk 3/7 0s
fetched chunk 4/7 0s
fetched chunk 5/7 0s
fetched chunk 6/7 0s
fetched chunk 7/7 0s
Data loaded 0s


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2025-01-02,1.006571,0.000000,0.004296,0.0,-0.009436,0.080374,0.000345,0.0,18.0,0.000842,1.0
2025-01-03,1.006571,0.000000,0.004296,0.0,-0.009436,0.080365,0.000345,0.0,18.0,0.000841,1.0
2025-01-06,1.006571,0.000000,0.004295,0.0,-0.009436,0.080357,0.000345,0.0,18.0,0.000841,1.0
2025-01-07,1.006571,0.000000,0.004295,0.0,-0.009436,0.080348,0.000345,0.0,18.0,0.000841,1.0
2025-01-08,1.013480,0.006864,0.004574,0.0,-0.009436,0.154234,0.000705,1.0,32.0,0.000841,1.0


# Common mistakes

Aggregate functions can bring knowledge from the future to the past. You can subtract the average or maximum value from each point in the time series. The maximum value may be in 2021, but how is this known from 2015?

```python
close = data.sel(field="close")

weights = close - close.max()
weights = close - close.min()
weights = close - close.std()
weights = close - close.sum()
```

Use future knowledge in the past

```python
weights = close.shift(time=-1)
```

Advance/Decline Line and Advance/Decline Ratio

```python
# no correct
close = data.sel(field="close")
weights = qnta.ad_line(close) * 1.0
weights = qnta.ad_ratio(close) * 1.0

# correct code
close = data.sel(field="close") * data.sel(field="is_liquid")
weights = qnta.ad_line(close) * 1.0
weights = qnta.ad_ratio(close) * 1.0

```

Standardization in price processing brings knowledge from the future to the past.

```python
def get_preprocessing(prices):
   from sklearn.preprocessing import StandardScaler
   scaler = StandardScaler()

   prices_pandas = prices.to_pandas()
   assets = data.coords["asset"].values
   for asset in assets:
       prices_pandas[asset] = scaler.fit_transform(prices_pandas[asset].values.reshape(-1, 1))
   return prices_pandas

prices = data.sel(field="close").fillna(0) * 1.0  # fill NaN
prices_standard_scaler = get_preprocessing(prices)
```

Quantile

```python
weights = (close < close.quantile(0.30, dim='asset'))*1
```

# How this template works

Runs your strategy multiple times cutting the data tail. It is similar to the step-by-step calculation.

By default, it runs 3 passes. It is enough to catch most errors. If you want to get more relevant results, you need to increase the number of passes. When you submit your strategy, the system runs about 1000 passes, which requires a very long time.

Also, this step performs some intermediate checks of the outputs.

When passes finish, you can find results in the folder `precheck_results`.

```python

evaluate_passes(passes=3, data_type=data_type)

# you can also set the the specific dates:
# evaluate_passes(dates=['2016-09-30', '2018-07-13', '2020-04-23'])
```

Assembles the output from the outputs calculated in the previous step.

```python
weights = assemble_output()
```

You can use a function to check the result

```python
check_output(weights, data_type=data_type)
```

